### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [11]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

import random

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [12]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [13]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [14]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [12]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '2252    Pleasantville (1998)',
 '1526    Hercules (1997)',
 '3817    Went to Coney Island on a Mission From God... ...',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [13]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [14]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [15]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [16]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1884    French Connection, The (1971)',
 '3458    Predator (1987)',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [78]:
class MatrixFactorization():
    def __init__(self):
        self.H = 0
    
    def similar_items(self, item_id, n=10):
        score = cosine_similarity(self.H.T)[item_id]
        return np.argsort(-score)[:n]
        
        

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [79]:
class SVD(MatrixFactorization):
    def __init__(self, regularization=0, learning_rate=1e-2, latent_size=64):
        self.regularization = regularization
        self.learning_rate = learning_rate
        self.latent_size = latent_size
        
    def fit(self, ratings, n_epoch=10):
        users = ratings["user_id"]
        movies = ratings["movie_id"]
        user_item = sp.coo_matrix((ratings['rating'], (users, movies)))
        V_csr = user_item.tocsr()
        
        self.W = np.random.uniform(0, 1/np.sqrt(self.latent_size), (user_item.shape[0], self.latent_size))
        self.H = np.random.uniform(0, 1/np.sqrt(self.latent_size), (self.latent_size, user_item.shape[1]))
        
        V = list(zip(users, movies))
        
        for i in range(n_epoch):
            random.shuffle(V)
            RMSE = 0
            for u, m in tqdm(V):
                err = (self.W[u, :] @ self.H[:, m]) - V_csr[u, m]
                self.W[u, :] = self.W[u, :] - self.learning_rate * (err * self.H[:, m] + self.regularization * self.W[u, :])
                self.H[:, m] = self.H[:, m] - self.learning_rate * (err * self.W[u, :] + self.regularization * self.H[:, m])
                RMSE += err ** 2
            RMSE = np.sqrt(RMSE / len(V))
            
            print(f'epoch: {i + 1}  RMSE: {RMSE}')
        
        self.I = self.W @ self.H
    

In [80]:
model = SVD()
model.fit(ratings, n_epoch=30)

100%|██████████| 1000209/1000209 [01:10<00:00, 14234.04it/s]


epoch: 1  RMSE: 1.3727164117743897


100%|██████████| 1000209/1000209 [01:07<00:00, 14740.53it/s]


epoch: 2  RMSE: 0.9396984315462857


100%|██████████| 1000209/1000209 [01:10<00:00, 14192.96it/s]


...

epoch: 28  RMSE: 0.5851126924824738


100%|██████████| 1000209/1000209 [01:09<00:00, 14337.62it/s]


epoch: 29  RMSE: 0.5829380523428344


100%|██████████| 1000209/1000209 [01:10<00:00, 14153.07it/s]


epoch: 30  RMSE: 0.5807353503692249


In [54]:
model.I = model.I.T

In [73]:
model.H.shape

(16, 3953)

In [82]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                        for x in model.similar_items(item_id, 10)]

In [83]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1661    Legal Deceit (1997)',
 '3221    Soft Toilet Seats (1999)',
 '1128    Line King: Al Hirschfeld, The (1996)',
 '935    My Man Godfrey (1936)',
 '3263    Legend of Lobo, The (1962)',
 '1509    War at Home, The (1996)',
 '584    Aladdin (1992)',
 "2286    Bug's Life, A (1998)"]

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [89]:
from numpy.linalg import inv

In [234]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [235]:
users = implicit_ratings["user_id"]

In [236]:
movies = implicit_ratings["movie_id"]

In [237]:
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))

In [238]:
user_item

<6041x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 575281 stored elements in COOrdinate format>

In [239]:
V_csr = user_item.tocsr()
R = V_csr

In [240]:
latent_size=64
U = np.random.uniform(0, 1/np.sqrt(latent_size), (user_item.shape[0], latent_size))
I = np.random.uniform(0, 1/np.sqrt(latent_size), (latent_size, user_item.shape[1]))

In [241]:
learning_rate=1e-2

In [242]:
res = inv(I @ I.T + learning_rate * np.eye(I.shape[0])) @ I

In [243]:
res.shape

(64, 3953)

In [160]:
res = inv(U.T @ U + learning_rate * np.eye(U.shape[1])) @ U.T

In [163]:
res.shape

(64, 6041)

In [191]:
b = np.array([[1,2,],[3,4]])

In [192]:
a = b.T
b

array([[1, 2],
       [3, 4]])

In [194]:
a[0] = [7,8]

In [233]:
R.T[1].shape

(1, 6041)

In [182]:
I = I.T
for i in range(I.shape[0]):
    I[i] = res @ R.T[i].toarray().squeeze()
I = I.T

In [170]:
res.shape

(64, 6041)

In [172]:
R.T[0].shape

(1, 6041)

In [178]:
(res @ R.T[0].toarray().squeeze()).shape

(64,)

In [181]:
I.shape

(64, 3953)

In [143]:
U.shape

(6041, 64)

In [133]:
res.shape

(3953, 64)

In [128]:
R.T.shape

(3953, 6041)

In [147]:
R.getrow(0)

<1x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [141]:
R.T[:,1].shape

(3953, 1)

In [142]:
res.T @ R.T[:,1]

array([[-0.01554302],
       [ 0.03042531],
       [ 0.04029914],
       [-0.05332524],
       [ 0.02038897],
       [ 0.02556518],
       [ 0.05140174],
       [ 0.00110859],
       [ 0.06405045],
       [ 0.05503139],
       [ 0.00851838],
       [-0.11906636],
       [-0.02538729],
       [ 0.01733274],
       [ 0.03060777],
       [ 0.08348502],
       [-0.0251294 ],
       [ 0.04759262],
       [-0.02274813],
       [-0.00273193],
       [-0.01388488],
       [-0.0219881 ],
       [ 0.01363473],
       [ 0.01801034],
       [-0.07736189],
       [ 0.06647598],
       [ 0.01092484],
       [ 0.00582614],
       [ 0.08779317],
       [-0.04310975],
       [-0.02866393],
       [-0.05621966],
       [-0.01131179],
       [ 0.04977502],
       [ 0.02452997],
       [ 0.03564118],
       [ 0.03491419],
       [ 0.03387444],
       [-0.04354998],
       [-0.01277083],
       [ 0.01480875],
       [ 0.06253637],
       [ 0.00155211],
       [-0.01176628],
       [-0.0530452 ],
       [-0

In [202]:
user_item.nonzero()

(array([   1,    1,    1, ..., 6040, 6040, 6040], dtype=int32),
 array([1193, 3408, 2355, ...,  562, 1096, 1097], dtype=int32))

In [226]:
class ALS(MatrixFactorization):
    def __init__(self, learning_rate=1e-5, latent_size=64):
        self.learning_rate = learning_rate
        self.latent_size = latent_size
        self.U = None
        self.I = None
        self.R = None
        
        
    
    def update_users(self):
        res = inv(self.I @ self.I.T + self.learning_rate * np.eye(self.I.shape[0])) @ self.I
        for i in range(self.U.shape[0]):
            self.U[i] = res @ self.R[i].T.toarray().squeeze()
        
        
    def update_items(self):
        res = inv(self.U.T @ self.U + self.learning_rate * np.eye(self.U.shape[1])) @ self.U.T
        I = self.I.T
        for i in range(self.I.shape[0]):
            I[i] = res @ self.R.T[i].toarray().squeeze()
        self.I = I.T

        
        
        
    def fit(self, ratings, n_epoch=10):
        users = ratings["user_id"]
        movies = ratings["movie_id"]
        user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
        V_csr = user_item.tocsr()
        self.R = V_csr
        
        self.U = np.random.uniform(0, 1/np.sqrt(self.latent_size), (user_item.shape[0], self.latent_size))
        self.I = np.random.uniform(0, 1/np.sqrt(self.latent_size), (self.latent_size, user_item.shape[1]))
        
        for i in range(n_epoch):
            self.update_users()
            self.update_items()
            
            y_pred = (self.U @ self.I)[user_item.nonzero()]
            y_true = user_item.data
            
            rmse = np.sqrt(((y_pred - y_true) ** 2).mean())
            print(f'Epoch: {i + 1} RMSE: {rmse}')

In [218]:
implicit_ratings

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
...,...,...,...
1000202,6040,1089,4
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [227]:
model = ALS()
model.fit(implicit_ratings, n_epoch=30)

Epoch: 1 RMSE: 0.9264155218618185
Epoch: 2 RMSE: 0.9222882504935315
Epoch: 3 RMSE: 0.9223700893809682
Epoch: 4 RMSE: 0.9226565868996359
Epoch: 5 RMSE: 0.922877924586063
Epoch: 6 RMSE: 0.9230402590526605
Epoch: 7 RMSE: 0.9231625630437792
Epoch: 8 RMSE: 0.9232568671491286
Epoch: 9 RMSE: 0.9233310728126807
Epoch: 10 RMSE: 0.9233905719491384
Epoch: 11 RMSE: 0.9234390942919102
Epoch: 12 RMSE: 0.923479259228855
Epoch: 13 RMSE: 0.9235129413989875
Epoch: 14 RMSE: 0.9235415093413724
Epoch: 15 RMSE: 0.9235659813601634
Epoch: 16 RMSE: 0.9235871286130877


KeyboardInterrupt: 

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных